# River meandering sonification

Sonification of outputs from a simple numerical model of river meandering.

See https://github.com/zsylvester/meanderpy

In [ ]:
import ipytone
import ipycanvas
import meanderpy as mp
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
t = ipytone.transport

## Run model

In [ ]:
nit = 10000                   # number of iterations
W = 200.0                    # channel width (m)
D = 6.0                      # channel depth (m)
depths = D * np.ones((nit,))  # channel depths for different iterations  
pad = 100                    # padding (number of nodepoints along centerline)
deltas = 50.0                # sampling distance along centerline           
Cfs = 0.011 * np.ones((nit,)) # dimensionless Chezy friction factor
crdist = 2 * W               # threshold distance at which cutoffs occur
kl = 60.0/(365*24*60*60.0)   # migration rate constant (m/s)
kv =  1.0e-12               # vertical slope-dependent erosion rate constant (m/s)
dt = 2*0.05*365*24*60*60.0     # time step (s)
dens = 1000                  # density of water (kg/m3)
saved_ts = 20                # which time steps will be saved
n_bends = 33                 # approximate number of bends you want to model
Sl = 0.0                     # initial slope (matters more for submarine channels than rivers)
t1 = 500                    # time step when incision starts
t2 = 500                    # time step when lateral migration starts
t3 = 1200                   # time step when aggradation starts
aggr_factor = 2e-9         # aggradation factor (m/s, about 0.18 m/year, it kicks in after t3)

In [ ]:
ch = mp.generate_initial_channel(W, D, Sl, deltas, pad, n_bends)
chb = mp.ChannelBelt(
    channels=[ch],
    cutoffs=[],
    cl_times=[0.0],
    cutoff_times=[]
)

In [ ]:
chb.migrate(nit,saved_ts,deltas,pad,crdist,depths,Cfs,kl,kv,dt,dens,t1,t2,t3,aggr_factor)

### Post-processing and utilities

Get domain / time extent

In [ ]:
xmin = np.min(chb.channels[0].x)
xmax = np.max(chb.channels[0].x)
ymax = 0
for i in range(len(chb.channels)):
    ymax = max(ymax, np.max(np.abs(chb.channels[i].y)))
ymax = ymax+2*chb.channels[0].W # add a bit of space on top and bottom
ymin = -1*ymax

In [ ]:
ct_times = np.array(chb.cutoff_times)
cl_times = np.array(chb.cl_times)

Compute channel and cutoff sinuosity

In [ ]:
def compute_sinuosity(channel, cutoff=False, outliers=False):
    x = channel.x
    y = channel.y
    z = channel.z
    
    if cutoff:
        x = x[0]
        y = y[0]
        z = z[0]

    dx, dy, dz, ds, s = mp.compute_derivatives(x,y,z)

    sinuosity = s[-1] / (x[-1] - x[0])
    
    if outliers:
        sinuosity = min(sinuosity, 40)
        sinuosity = max(sinuosity, 2)
    
    return sinuosity

In [ ]:
sinuosities = []
y_mean = []

for channel in chb.channels:
    sinuosities.append(compute_sinuosity(channel))
    y_mean.append(channel.y.mean())


In [ ]:
ct_sinuosities = []
ct_x_mean = []


for cutoff in chb.cutoffs:
    ct_sinuosities.append(
        compute_sinuosity(cutoff, cutoff=True, outliers=True)
    )
    ct_x_mean.append(cutoff.x[0].mean())


## Setup canvas (drawing)

In [ ]:
def get_points(channel, banks=False, cutoff=False):
    x = channel.x
    y = channel.y
    
    if cutoff:
        x = x[0]
        y = y[0]
    
    if banks:
        xm, ym = mp.get_channel_banks(x, y, -1)
        x = np.reshape(xm, (2, x.size))
        y = np.reshape(ym, (2, y.size))

    x_pix = (x - xmin) / (xmax - xmin) * width
    y_pix = (y - ymin) / (ymax - ymin) * height 

    if banks:
        return [np.stack((x_pix[i], y_pix[i]), axis=1) for i in range(2)]
    else:
        return np.stack((x_pix, y_pix), axis=1)


In [ ]:
op2hex_map = {
    100: "FF",
    95: "F2",
    90: "E6",
    85: "D9",
    80: "CC",
    75: "BF",
    70: "B3",
    65: "A6",
    60: "99",
    55: "8C",
    50: "80",
    45: "73",
    40: "66",
    35: "59",
    30: "4D",
    25: "40",
    20: "33",
    15: "26",
    10: "1A",
    5: "0D",
    0: "00",
}


def opacity2color(opacity):
    if opacity < 0:
        opacity = 0
    cstr = op2hex_map[5 * round(opacity * 100 / 5)]
    return "#" + cstr * 3
    

In [ ]:
width = 1200
height = 400

mcanvas = ipycanvas.MultiCanvas(n_canvases=2, width=width, height=height)

# background
mcanvas[0].fill_style = "black"
mcanvas[0].rough_fill_style = "solid"
mcanvas[0].fill_rect(0, 0, width, height)


In [ ]:
def draw_step(step):
    canvas = mcanvas[1]
    
    current_time = cl_times[step]
    
    with ipycanvas.hold_canvas(canvas):
        canvas.clear()

        # draw river banks
        max_banks = 20
        channel_indices = np.arange(max(0, step - max_banks), step)
        channel_opacity = np.linspace(0, 0.8, channel_indices.size)

        canvas.line_width = 0.8

        for i, idx in enumerate(channel_indices - 1):
            canvas.stroke_style = opacity2color(channel_opacity[i])
            points = get_points(chb.channels[idx], banks=True)
            canvas.stroke_lines(points[0])
            canvas.stroke_lines(points[1])

        # draw oxbow lakes
        tresh = 50  # rel time threshold
        rel_time = ct_times - current_time
        mask = np.logical_and(rel_time < 0, rel_time > -tresh)
        cutoffs = np.array(chb.cutoffs)[mask]
        cutoff_opacity = (rel_time[mask] + tresh) / tresh * 0.4

        canvas.line_width = 8

        for i, cutoff in enumerate(cutoffs):
            canvas.stroke_style = opacity2color(cutoff_opacity[i])
            points = get_points(cutoff, cutoff=True)
            canvas.stroke_lines(points)

        # draw main channel
        points = get_points(chb.channels[channel_indices[-1]])
        canvas.stroke_style = "white"
        canvas.line_width = 8
        canvas.stroke_lines(points)


## Setup sound

Clip duration:

In [ ]:
duration = 50

Monophonic synth used to sonify the main river channel, connected to:

- a filter controled by the sinuosity of the main river channel.
- a panner controled by the mean y-coordinates of the main river channel.

In [ ]:
pan = ipytone.Panner()
filtr = ipytone.Filter()
synth = ipytone.PolySynth(volume=-5)
synth.chain(filtr, pan, ipytone.destination)

In [ ]:
synth.voice.oscillator.type = "fmsine"
filtr.q.value = 10

In [ ]:
synth_freqs = np.array(sinuosities) * 4000 - 3900
synth_pans = np.array(y_mean) / np.max(np.abs(y_mean)) * -0.5


def synth_automation(time):
    synth.trigger_attack_release(["C2", "C3", "E3", "G3"], duration, time=time)
    filtr.frequency.set_value_curve_at_time(synth_freqs, time, duration)
    pan.pan.set_value_curve_at_time(synth_pans, time, duration)


synth_eid = t.schedule(synth_automation, 0)

Polyphonic synths playing high notes at each river cutoff event (two synths will generate notes on the left and right stereo channels, the synth used will depend on to the position of the cutoff event along the x-axis).

In [ ]:
delay = ipytone.FeedbackDelay().to_destination()

mpan1 = ipytone.Panner(pan=-0.85)
msynth1 = ipytone.PolySynth(volume=-14)
msynth1.chain(mpan1, delay)

mpan2 = ipytone.Panner(pan=0.85)
msynth2 = ipytone.PolySynth(volume=-14)
msynth2.chain(mpan2, delay)

In [ ]:
msynth1.voice.oscillator.type = "sine"
msynth2.voice.oscillator.type = "sine"
msynth1.voice.envelope.release = 2
msynth2.voice.envelope.release = 2

In [ ]:
delay.wet.value = 0.2
delay.feedback.value = 0.15

In [ ]:
ct_play_times = ct_times / ct_times[-1] * duration
msynth_freqs = np.array(ct_sinuosities) * 100 - 90
msynth_pans = np.array(ct_x_mean) / xmax - 0.5

In [ ]:
def clb1(time, note):
    msynth1.trigger_attack_release(note.note, 0.1, time=time)
    
def clb2(time, note):
    msynth2.trigger_attack_release(note.note, 0.1, time=time)

In [ ]:
A4 = 440
C0 = A4 * np.power(2, -4.75)
name = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
    
def pitch_cmaj(freq):
    """Return a note on the Cmaj scale from any frequency."""
    h = round(12 * np.log2(freq / C0))
    octave = h // 12
    n = h % 12
    note_cmaj = name[n].replace('#', '')
    return note_cmaj + str(octave)


In [ ]:
mpart1 = ipytone.Part(
    callback=clb1,
    events=[
        {"note": pitch_cmaj(freq), "time": time}
        for freq, time, pan in zip(msynth_freqs, ct_play_times, msynth_pans)
        if pan < 0
    ],
)

mpart2 = ipytone.Part(
    callback=clb2,
    events=[
        {"note": pitch_cmaj(freq), "time": time}
        for freq, time, pan in zip(msynth_freqs, ct_play_times, msynth_pans)
        if pan >= 0
    ],
)

Canvas update (more or less) in-sync with the audio.

In [ ]:
def update_canvas(change):
    time = change["new"]
    step = int(time / duration * cl_times.size) 
    if step == 0:
        step = 1
    draw_step(step)
    

In [ ]:
t.schedule_observe(
    update_canvas, update_interval=0.05, name="seconds", transport=True
)

## Play it!

In [ ]:
mcanvas

In [ ]:
mpart1.start().stop(f"+{duration}")
mpart2.start().stop(f"+{duration}")
t.start().stop(f"+{duration}")

## Clean-up

In [ ]:
t.schedule_unobserve(update_canvas)

In [ ]:
t.cancel()
mpart1.dispose()
mpart2.dispose()

In [ ]:
synth.dispose()
filtr.dispose()
pan.dispose()
msynth1.dispose()
mpan1.dispose()
msynth2.dispose()
mpan2.dispose()
delay.dispose()

In [ ]:
import pickle

In [ ]:
with open("run01.pkl","wb") as file:
    pickle.dump(chb, file)

In [ ]:
with open(r"run01.pkl", "rb") as file:
    chb = pickle.load(file)